In [1]:
# Author : Trong Canh Nguyen

# This script considers all the products a user has ordered
#
# We train a model computing the probability of reorder on the "train" data
#
# For the submission, we keep the orders that have a probability of
# reorder higher than a threshold


import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
from helper import *
IDIR = '../input/'
FEATURES_PATH = './features3/'

In [ ]:
gc.collect()

## Data Load

In [ ]:
#data = pd.read_csv(FEATURES_PATH + "data.csv", dtype= dtype_dict)

In [ ]:
data = pd.read_hdf(FEATURES_PATH + "data.h5", "data")
data.reset_index(inplace=True)
print("memory = ", data.memory_usage().sum()/1000000)

In [ ]:
data.head()

In [ ]:
columns = list(data.columns)
not_features = ['user_id', 'product_id', 'aisle_id',  'user_eval_set', 'up_reordered']
features = list(set(columns) - set(not_features))
print("number of features", len(features))

features

## Product embedding

In [ ]:
product_embedding = pd.read_csv(IDIR+"product_embeddings.csv")
product_embedding.drop(['product_name','aisle_id','department_id'], axis = 1, inplace = True)
columns_dict = dict([(str(i), 'pe_'+str(i)) for i in range(32)])
product_embedding.rename(columns=columns_dict, inplace = True)
pe_columns = ['pe_'+str(i) for i in range(32)]
product_embedding[pe_columns] = product_embedding[pe_columns].astype(np.float32)

In [ ]:
data = data.merge(product_embedding, on="product_id", how = "left")

In [ ]:
print("memory = ", data.memory_usage().sum()/1000000)

In [ ]:
data.set_index(['user_id', 'product_id'], inplace = True)

In [ ]:
data.to_hdf(FEATURES_PATH + "data_pe.h5", "data", mode = 'a')

# MODEL

In [ ]:
#data_features = data[data.user_eval_set == "train"][['user_id', 'product_id']+ features + ["up_reordered"]]
data_features = data[data.user_eval_set == "train"]

tmp = data_features.user_id.unique()
user_train = tmp[0:120000]
user_valid = tmp[120000:]

data_train = data_features[data_features.user_id.isin(user_train)]
data_valid = data_features[data_features.user_id.isin(user_valid)]
print(len(data_train))
print(len(data_valid))

In [ ]:
len(data_features)

In [ ]:
#data_features.to_hdf(FEATURES_PATH + "lgb_data.h5", "data_features", mode="a")
data_train.to_hdf(FEATURES_PATH + "lgb_data.h5", "data_train", mode="a")
data_valid.to_hdf(FEATURES_PATH + "lgb_data.h5", "data_valid", mode="a")

In [ ]:
data_test = data[data.user_eval_set == "test"]
data_test.to_hdf(FEATURES_PATH + "lgb_data.h5", "data_test", mode="a")

## Load data_train, data_valid

In [4]:
features = ['up_order_period',
 'user_dep_ratio',
 'up_order_rate',
 'up_order_period_mean',
 'order_hour_of_median',
 'user_dep_reordered_ratio',
 'up_first_order',
 'user_aisle_reordered_ratio',
 'aisle_reorder_ratio',
 'up_last_order',
 'add_to_cart_order_relative',
 'up_orders_since_last_order',
 'user_total_order',
 'up_add_to_cart_order_mean',
 'order_hour_of_day_mean',
 'user_order_size_mean',
 'user_order_hour_of_day_mean',
 'is_organic',
 'user_order_dow',
 'user_reorder_rate',
 'up_days_since_last_order',
 'user_order_hour_of_day',
 'product_reorder_probability',
 'department_id',
 'user_days_since_prior_mean',
 'user_aisle_ratio',
 'up_order_dow_median',
 'product_reorder_ratio',
 'up_orders',
 'up_order_dow_mean',
 'dep_reorder_ratio',
 'add_to_cart_order_relative_median',
 'up_days_since_prior_order_mean',
 'user_order_dow_mean',
 'up_recency',
 'up_order_rate_since_first_order',
 'user_days_since_prior_order']

#features = features + pe_columns

print(len(features))

37


In [2]:
data_train = pd.read_hdf(FEATURES_PATH + "lgb_data.h5", "data_train")
data_valid = pd.read_hdf(FEATURES_PATH + "lgb_data.h5", "data_valid")
print("data_train len", len(data_train))
print("data_valid len", len(data_valid))

#columns = list(data_train.columns)
#not_features = ['user_id', 'product_id', 'up_reordered']
#features = list(set(columns) - set(not_features))
#print("number of features", len(features))
#features

data_train len 7757907
data_valid len 716754


In [ ]:
#data_train = data_train.merge(product_embedding, on='product_id', how = 'left')
#data_valid = data_valid.merge(product_embedding, on='product_id', how = 'left')

In [ ]:
len(data_train.columns)

In [ ]:
gc.collect()

In [5]:
X_train= data_train[features].values
y_train= data_train['up_reordered'].values
X_valid= data_valid[features].values
y_valid= data_valid['up_reordered'].values

In [5]:
del data_train
del data_valid
gc.collect()

80

In [6]:
import h5py
#with h5py.File(FEATURES_PATH+ 'data_train_product_embedding.h5', 'w') as hf:
with h5py.File(FEATURES_PATH+ 'data_train.h5', 'w') as hf:
    hf.create_dataset("X_train",  data=X_train)
    hf.create_dataset("y_train",  data=y_train)

#with h5py.File(FEATURES_PATH+ 'data_train_product_embedding.h5', 'a') as hf:
with h5py.File(FEATURES_PATH+ 'data_train.h5', 'a') as hf:
    hf.create_dataset("X_valid",  data=X_valid)
    hf.create_dataset("y_valid",  data=y_valid)
   

In [4]:
import h5py
with h5py.File(FEATURES_PATH+ 'data_train.h5', 'r') as hf:
    X_train = hf['X_train'][:]
    y_train = hf['y_train'][:]

In [ ]:
weight_train = 1 - (1- y_train)*0.8
weight_valid = 1 - (1- y_valid)*0.8

In [6]:
print('formating for lgb')
d_train = lgb.Dataset(X_train, label=y_train, feature_name = features, categorical_feature='department_id')
d_valid = lgb.Dataset(X_valid, label=y_valid, feature_name = features, categorical_feature='department_id', reference=d_train)
#d_train.save_binary(FEATURES_PATH +  'train.bin')
#d_valid.save_binary(FEATURES_PATH +  'valid.bin')

formating for lgb


In [12]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 512,
    'max_depth': 12,
    'min_data_in_leaf':1000,
    'feature_fraction': 0.7,
    #'bagging_fraction': 0.95,
    #'bagging_freq': 5,
    'learning_rate':0.05
}
ROUNDS = 350

print('light GBM train :-)')
bst = lgb.train(params, d_train, ROUNDS)

In [13]:
print('light GBM train :-)')
bst = lgb.train(params, d_train, ROUNDS)

light GBM train :-)


In [125]:
bst.save_model(FEATURES_PATH + 'lgb/trained_model_37_features_2.txt')

In [ ]:
#bst = lgb.Booster(model_file=FEATURES_PATH+ 'lgb/trained_model_28_features_categorical.txt')

In [15]:
bst.add_valid(d_valid, "valid")

In [16]:
bst.eval_train()

[('training', 'binary_logloss', 0.23381499168485428, False)]

In [17]:
bst.eval_valid()

[('valid', 'binary_logloss', 0.24296611589068384, False)]

## XGBOOST

In [5]:
import xgboost

/home/canh/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
d_train = xgboost.DMatrix(X_train, y_train)
xgb_params = {
    "objective"         : "reg:logistic"
    ,"eval_metric"      : "logloss"
    ,"eta"              : 0.1
    ,"max_depth"        : 6
    ,"min_child_weight" :10
    ,"gamma"            :0.70
    ,"subsample"        :0.76
    ,"colsample_bytree" :0.95
    ,"alpha"            :2e-05
    ,"lambda"           :10
}

watchlist= [(d_train, "train")]
bst = xgboost.train(params=xgb_params, dtrain=d_train, num_boost_round=80, evals=watchlist, verbose_eval=10)
xgboost.plot_importance(bst)

## Train on all dataset

In [ ]:
X_all = data_features[features]
y_all = data_features['up_reordered']

In [ ]:
dataset_all = lgb.Dataset(X_all, label=y_all)

In [ ]:
print('light GBM train :-)')
bst_all = lgb.train(params, dataset_all, ROUNDS)

In [ ]:
bst_all.save_model(FEATURES_PATH + 'lgb/trained_model_num_leaves500_ALL.txt')

In [ ]:
bst_all.eval_train()

### Validation

In [ ]:
def precision(y, y_, correct, has_none=False):
    if y_ > 0:
        if not has_none:
            return correct / y_
        else:
            if y > 0:
                return correct / (y_ + 1)
            else:
                return 1. / (y_ + 1)
    else:
        return 1.0


def recall(y, y_, correct, has_none=False):
    if y > 0:
        return correct / y
    else:
        if has_none or (y_ == 0):
            return 1.0
        else:
            return 0.


def f1(y, y_, correct, has_none=False):
    p = precision(y, y_, correct, has_none)
    r = recall(y, y_, correct, has_none)
    if (p == 0) and (r == 0):
        return 0.
    f1 = 2 * p * r / (p + r)
    return f1


def compute_f1(valid_df, threshold):
    valid_df['y_'] = valid_df['pred'] > threshold
    valid_df['correct'] = (valid_df['y'] == valid_df['y_']) & (valid_df['y_'])
    result = valid_df.groupby('user_id').sum()
    result['f1'] = result.apply(lambda row: f1(row['y'], row['y_'], row['correct']), axis=1)
    return result['f1'].mean()

In [ ]:
#bst = lgb.Booster(model_file=FEATURES_PATH+ 'lgb/trained_model_27_features_3.txt')

In [18]:
pred_valid = bst.predict(X_valid)
pred_valid

array([ 0.04016588,  0.05939614,  0.06182783, ...,  0.1355507 ,
        0.0247439 ,  0.12688935])

In [20]:
valid_df = data_valid[['user_id', 'product_id']].copy()
valid_df["y"] = y_valid
valid_df["pred"] = pred_valid
valid_df["y_"] = valid_df["pred"]  >= 0.2
valid_df['correct'] = (valid_df['y'] == valid_df['y_']) & (valid_df['y_'])
valid_df.sort_values(['user_id', 'pred'], ascending=[True, False], inplace = True)
#print("valid log loss = ", -((valid_df["y"]*np.log(valid_df["pred"])+ (1.-valid_df["y"])* np.log(1.- valid_df["pred"]))).mean())

In [21]:
compute_f1(valid_df, 0.20)

0.37876167530883526

In [ ]:
valid_df

In [ ]:
result = valid_df.groupby('user_id').sum()
result['precision'] = result.apply(lambda row: precision(row['y'], row['y_'], row['correct']), axis=1)
result['recall'] = result.apply(lambda row: recall(row['y'], row['y_'], row['correct']), axis=1)
result['f1'] = result.apply(lambda row: f1(row['y'], row['y_'], row['correct']), axis=1)

print("precision mean = ", result.precision.mean())
print("recall mean = ", result.recall.mean())
print("f1 mean = ", result.f1.mean())

In [ ]:
result

In [ ]:
#valid_df.to_csv(FEATURES_PATH+ "valid_df.csv")
valid_df.to_hdf(FEATURES_PATH+ "results.h5", "valid_df", mode = "a")

In [ ]:
valid_df.head()

## Generate predictions

In [ ]:
#bst_all = lgb.Booster(model_file=FEATURES_PATH+ 'trained_model.txt')

In [ ]:
data_test = pd.read_hdf(FEATURES_PATH + "lgb_data.h5", "data_test")

In [ ]:
print('light GBM predict')
pred_test = bst.predict(data_test[features])
pred_test

In [ ]:
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32},
        usecols=["order_id", "user_id", "eval_set"])

test_orders= orders[orders.eval_set == 'test']

In [ ]:
prediction = pd.DataFrame()
prediction[['user_id', 'product_id']] = data_test[['user_id', 'product_id']]
prediction['proba'] = pred_test
prediction.sort_values(by=['user_id', 'proba'], ascending=[True, False], inplace=True)
prediction = pd.merge(prediction, test_orders[['order_id', 'user_id']], on="user_id", how='left')

In [ ]:
prediction

## Feature importance

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print('Plot feature importances...')
ax = lgb.plot_importance(bst, max_num_features=50)
plt.show()


### Recommendation using threshold

In [ ]:
threshold = 0.20
recommend = prediction[prediction.proba >= threshold].groupby('order_id').product_id.apply(list)

In [ ]:
recommend.head()

In [ ]:
add_none_df = prediction[prediction.proba >= threshold].groupby('order_id').proba.agg([np.size, np.max])
add_none_df['None'] = (add_none_df['size'] > 0) & (add_none_df['size'] < 4) & (add_none_df['amax'] < 0.46)
add_none_df.head()

In [ ]:
recommend_df = pd.DataFrame()
recommend_df["count"] = prediction.groupby('order_id').size()
recommend_df['product_list'] = recommend
recommend_df['none'] = add_none_df['None']


In [ ]:
recommend_df.head()

In [ ]:
def generate_prediction(row):
    p = row.product_list
    if type(p) == list:
        result = ' '.join([str(x)  for x in p])
        
        if row.none:
            result = 'None ' + result
        
        return result
    else:
        return 'None'

In [ ]:
recommend_df['products']= recommend_df.apply(lambda row:  generate_prediction(row) , axis = 1) 

In [ ]:
recommend_df['products']

In [ ]:
recommend_df['products'].to_csv(FEATURES_PATH +  'lgb/recommend5_none.csv', header = True)

### Recommendation using average user basket

In [ ]:
products = []
count = 0
for _,row in test_orders[['user_id', 'order_id']].iterrows():
    count += 1
    if (count)%10000 == 0:
        print(count)    
    
    user_id, order_id = row['user_id'], row['order_id']
    n = int(user_basket_avg.ix[user_id].basket_size_avg)+1
    products.append(list(prediction[prediction.user_id == user_id].product_id[:n]))
    
# create submission
submission = pd.DataFrame()
submission['order_id'] = test_orders['order_id']
submission['products'] = [' '.join([str(x) for x in p]) for p in products]
submission.sort_values(by='order_id', inplace = True)
submission.to_csv(FEATURES_PATH + 'submission.csv', index=False)

## Investigate result

In [ ]:
valid_df.head()

In [ ]:
tmp

In [ ]:
users_none_incorrect = tmp[(tmp.y == 0) &  (tmp.y_ > 0)].user_id

In [ ]:
valid_df[valid_df.user_id.isin(users_none_incorrect) & valid_df.y_].pred.mean()

In [ ]:
valid_df[valid_df.y_].pred.mean()

In [ ]:
print("None predicted correctly = ", len(tmp[(tmp.y == 0) &  (tmp.y_ == 0)]) / len(tmp[tmp.y == 0]))

In [ ]:
print("precision mean = ", result.precision.mean())
print("recall mean = ", result.recall.mean())
print("f1 mean = ", result.f1.mean())

In [ ]:
p = 0.3934358303379222
r = 0.5387833182445639
2*p*r/(p+r)

In [ ]:
print("Not None but predict None = ", len(tmp[(tmp.y > 0) &  (tmp.y_ == 0)]) / len(tmp[tmp.y > 0]))
print("None but predicted not None = ", len(tmp[(tmp.y == 0) &  (tmp.y_ > 0)]) / len(tmp[tmp.y == 0]))

In [ ]:
users_valid_not_reorder = tmp[(tmp.y == 0)].reset_index().user_id

In [ ]:
data_valid_user_not_reorder = data_valid[data_valid.user_id.isin(users_valid_not_reorder)]

In [ ]:
data_valid_user_not_reorder.head()

In [ ]:
user_info = pd.read_hdf(FEATURES_PATH + "features.h5", "user_info").reset_index()

In [ ]:
user_info_user_not_reorder = user_info[user_info.user_id.isin(users_valid_not_reorder)]

In [ ]:
user_info_user_not_reorder

In [ ]:
trains = pd.read_hdf(IDIR + "input.h5", "trains")
trains = trains.merge(orders, on= "order_id", how="left")

In [ ]:
trains[trains.user_id == 188603]

In [ ]:
valid_df[valid_df.user_id == 188603]

In [ ]:
def f1_score(p,q):
    return 2*p*q/(p+q)

In [ ]:
f1_score(1, 0.5)

In [ ]:
tmp[tmp.y == 0].y_.mean()

### When there is None, we predict on average 2.8542780748663104 products ==> can be improved

In [ ]:
valid_df = data_valid[['user_id', 'product_id']].copy()
valid_df["y"] = y_valid
valid_df["pred"] = pred_valid

In [ ]:
valid_df["y_"] = valid_df["pred"]  >= 0.2
valid_df['correct'] = (valid_df['y'] == valid_df['y_']) & (valid_df['y_'])
valid_df.sort_values(['user_id', 'pred'], ascending=[True, False], inplace = True)
valid_df.head()

In [22]:
threshold_none = 0.45
result = valid_df.groupby('user_id').agg({'y':np.sum,'y_':np.sum, 'correct':np.sum, 'pred':np.max}).reset_index()
result['None'] = (result['y_'] > 0) & (result['y_'] < 4) &(result['pred'] < threshold_none)
result['precision'] = result.apply(lambda row: precision(row['y'], row['y_'], row['correct'], row['None']), axis=1)
result['recall'] = result.apply(lambda row: recall(row['y'], row['y_'], row['correct'], row['None']), axis=1)
result['f1'] = result.apply(lambda row: f1(row['y'], row['y_'], row['correct'], row['None']), axis=1)
print(result.f1.mean())

0.38652892812353923


In [28]:
rsult.y.value_counts()

NameError: name 'rsult' is not defined

In [24]:
data_valid = data_valid.merge(result, on=['user_id'], how='left')

In [37]:
result[(result.y > 0) & (result.correct == 0)].y_.value_counts()

0.0     403
1.0     361
2.0     307
3.0     224
4.0     144
5.0      97
6.0      65
7.0      46
8.0      38
10.0     33
9.0      20
14.0     10
13.0      9
12.0      9
11.0      8
16.0      7
15.0      7
21.0      4
20.0      3
30.0      1
34.0      1
24.0      1
18.0      1
17.0      1
19.0      1
Name: y_, dtype: int64

In [35]:
len(result)

11209

In [36]:
len(result[(result.y > 0) & (result.y_ == 1)])

674

In [41]:
valid_df.set_index(['user_id', 'product_id'], inplace = True)

In [43]:
data_valid.set_index(['user_id', 'product_id'], inplace = True)

In [44]:
data_valid.columns

Index(['up_order_dow_mean', 'up_order_dow_median', 'order_hour_of_day_mean',
       'order_hour_of_median', 'add_to_cart_order_relative',
       'add_to_cart_order_relative_median', 'up_orders', 'up_first_order',
       'up_last_order', 'up_order_rate', 'up_orders_since_last_order',
       'up_order_rate_since_first_order', 'up_days_since_last_order',
       'up_order_period', 'up_order_period_mean',
       'up_days_since_prior_order_mean', 'up_add_to_cart_order_mean',
       'up_reordered', 'user_total_order', 'user_reorder_rate',
       'user_order_size_mean', 'user_days_since_prior_mean', 'user_order_dow',
       'user_order_hour_of_day', 'user_eval_set',
       'user_days_since_prior_order', 'user_order_dow_mean',
       'user_order_hour_of_day_mean', 'product_reorder_ratio', 'aisle_id',
       'department_id', 'product_reorder_probability', 'is_organic',
       'aisle_reorder_ratio', 'dep_reorder_ratio',
       'user_aisle_reordered_ratio', 'user_aisle_ratio',
       'user_dep_reo

In [45]:
valid_df['mean_reorder_size'] = data_valid['user_reorder_rate']*data_valid['user_order_size_mean']

In [46]:
valid_df.head()

y      pred     y_  correct  mean_reorder_size
user_id product_id                                                
188560  46676       0  0.230936   True    False               0.75
        35140       0  0.132307  False    False               0.75
        12932       0  0.128439  False    False               0.75
        27104       0  0.116702  False    False               0.75
        33120       0  0.112261  False    False               0.75

In [47]:
alpha = 0
beta = 0.2

In [76]:
def compute_f1_bis(valid_df, alpha, beta_low, beta_high):
    valid_df['y_'] = (valid_df['mean_reorder_size'] < alpha) & (valid_df['pred'] > beta_low) | (valid_df['mean_reorder_size'] >= alpha) & (valid_df['pred'] > beta_high) 
    valid_df['correct'] = (valid_df['y'] == valid_df['y_']) & (valid_df['y_'])
    result = valid_df.groupby('user_id').sum()
    result['f1'] = result.apply(lambda row: f1(row['y'], row['y_'], row['correct']), axis=1)
    return result['f1'].mean()

In [100]:
compute_f1_bis(valid_df, 0.0, 0.05, 0.2)

0.37876167530883526

In [70]:
compute_f1_bis(valid_df, 0.0001, 0.20)

0.37881612759673244

In [102]:
pred_train = bst.predict(X_train)

In [110]:
gc.collect()

294

In [105]:
train_df = data_train[['user_id', 'product_id']].copy()
train_df["y"] = y_train
train_df["pred"] = pred_train
train_df["y_"] = train_df["pred"]  >= 0.2
train_df['correct'] = (train_df['y'] == train_df['y_']) & (train_df['y_'])
train_df.sort_values(['user_id', 'pred'], ascending=[True, False], inplace = True)
train_df.head()

,user_id,product_id,y,pred,y_,correct
0,1,196,1,0.891531,True,True
3,1,12427,0,0.853975,True,False
1,1,10258,1,0.837507,True,True
8,1,25133,1,0.774875,True,True
16,1,46149,1,0.735626,True,True


In [118]:
threshold_none = 0.45
result_train = train_df.groupby('user_id').agg({'y':np.sum,'y_':np.sum, 'correct':np.sum, 'pred':np.max}).reset_index()
result_train['None'] = (result_train['y_'] > 0) & (result_train['y_'] < 4) &(result_train['pred'] < threshold_none)
result_train['precision'] = result_train.apply(lambda row: precision(row['y'], row['y_'], row['correct'], row['None']), axis=1)
result_train['recall'] = result_train.apply(lambda row: recall(row['y'], row['y_'], row['correct'], row['None']), axis=1)
result_train['f1'] = result_train.apply(lambda row: f1(row['y'], row['y_'], row['correct'], row['None']), axis=1)
print(result_train.f1.mean())

0.4077120668767288


In [119]:
false_user_id = result_train[result_train.f1 == 0].reset_index().user_id

In [120]:
result_train.to_hdf(FEATURES_PATH+"save.h5", "result_train")
train_df.to_hdf(FEATURES_PATH+"save.h5", "train_df")

In [121]:
false_data_train = data_train[data_train.user_id.isin(false_user_id)]

In [123]:
false_data_train.to_hdf(FEATURES_PATH+"save.h5", "false_data_train")

In [122]:
len(false_data_train)

1198272

In [124]:
X_false_train = false_data_train[features].values
y_false_train = false_data_train['up_reordered'].values

In [126]:
d_false_train = lgb.Dataset(X_false_train, label=y_false_train, feature_name = features, categorical_feature='department_id')

In [127]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 100,
    'max_depth': 12,
    'min_data_in_leaf':100,
    'feature_fraction': 0.7,
    #'bagging_fraction': 0.95,
    #'bagging_freq': 5,
    'learning_rate':0.05
}
ROUNDS = 100

print('light GBM train :-)')
bst2 = lgb.train(params, d_false_train, ROUNDS)

light GBM train :-)


In [128]:
pred_valid_false = bst2.predict(X_valid)

In [129]:
len(pred_valid_false)

716754

In [133]:
valid_df = data_valid.reset_index()[['user_id', 'product_id']].copy()
valid_df["y"] = y_valid
valid_df["pred"] = pred_valid_total
valid_df["y_"] = valid_df["pred"]  >= 0.2
valid_df['correct'] = (valid_df['y'] == valid_df['y_']) & (valid_df['y_'])
#valid_df.sort_values(['user_id', 'pred'], ascending=[True, False], inplace = True)

In [147]:
theta = 0.2
pred_valid_total = (1-theta)*pred_valid + theta*pred_valid_false
valid_df["pred"] = pred_valid_total

In [148]:
compute_f1(valid_df, 0.20)

0.3744995492294094